## Loading libraries

In [2]:
import os

import pretty_midi
import note_seq
from note_seq.midi_io import midi_to_note_sequence, note_sequence_to_pretty_midi

import IPython.display


## Defining parameters and functions

In [3]:
NOTE_LENGTH_16TH_120BPM = 0.25 * 60 / 120
BAR_LENGTH_120BPM = 4.0 * 60 / 120

In [10]:
def empty_note_sequence(qpm=120.0, total_time=0.0):
    note_sequence = note_seq.protobuf.music_pb2.NoteSequence()
    note_sequence.tempos.add().qpm = qpm
    note_sequence.ticks_per_quarter = note_seq.constants.STANDARD_PPQ
    note_sequence.total_time = total_time
    return note_sequence

def token_sequence_to_note_sequence(token_sequence, use_program=True, use_drums=True):

    if isinstance(token_sequence, str):
        token_sequence = token_sequence.split()

    note_sequence = empty_note_sequence()
    current_program = 1
    current_is_drum = False
    for token_index, token in enumerate(token_sequence):

        if token == "PIECE_START":
            pass
        elif token == "PIECE_END":
            print("The end.")
            break
        elif token == "TRACK_START":
            current_bar_index = 0
            pass
        elif token == "TRACK_END":
            pass
        elif token.startswith("INST"):
            current_instrument = token.split("=")[-1]
            if current_instrument != "DRUMS" and use_program:
                current_instrument = int(current_instrument)
                current_program = int(current_instrument)
                current_is_drum = False
            if current_instrument == "DRUMS" and use_drums:
                current_instrument = 0
                current_program = 0
                current_is_drum = True
        elif token == "BAR_START":
            current_time = current_bar_index * BAR_LENGTH_120BPM
            current_notes = {}
        elif token == "BAR_END":
            current_bar_index += 1
            pass
        elif token.startswith("NOTE_ON"):
            pitch = int(token.split("=")[-1])
            note = note_sequence.notes.add()
            note.start_time = current_time
            note.end_time = current_time + 4 * NOTE_LENGTH_16TH_120BPM
            note.pitch = pitch
            note.instrument = int(current_instrument)
            note.program = current_program
            note.velocity = 80
            note.is_drum = current_is_drum
            current_notes[pitch] = note
        elif token.startswith("NOTE_OFF"):
            pitch = int(token.split("=")[-1])
            if pitch in current_notes:
                note = current_notes[pitch]
                note.end_time = current_time
        elif token.startswith("TIME_DELTA"):
            delta = float(token.split("=")[-1]) * NOTE_LENGTH_16TH_120BPM
            current_time += delta
        elif token.startswith("DENSITY="):
            pass
        elif token == "[PAD]":
            pass
        else:
            assert False, token

    return note_sequence

def note_seq_to_text(notes):
    text = "PIECE_START"
    
    instrument = -1
    track_opened = False
    current_bar_end = 0
    
    for note in notes:
        current_instrument = note.instrument
        current_note = note.pitch
        current_start_time = note.start_time
        current_end_time = note.end_time
        
        # Bar_end?
        if current_start_time >= current_bar_end and track_opened:
            text += " BAR_END"
        
        if current_instrument != instrument:
            text += " TRACK_END" if track_opened else ""
            text += f" TRACK_START INST={current_instrument} DENSITY=0"
            instrument = current_instrument
            track_opened = True
            current_bar_end = 0

        
        # Bar_start?
        if current_start_time >= current_bar_end:
            text += " BAR_START"
            current_bar_end += 2.0
        
        current_timedelta = (current_end_time - current_start_time)*8
        #TODO multiple note_ons can be
        text += f" NOTE_ON={current_note} TIME_DELTA={current_timedelta} NOTE_OFF={current_note}"
        
    text += " BAR_END TRACK_END PIECE_END"
    return text

## Note Sequence to Text

In [15]:
# Load MIDI file into PrettyMIDI object. Give path to a midi file
midi_folder = "/home/karlos/Documents/workspace/js-fakes/midi"
midi_data = pretty_midi.PrettyMIDI(os.path.join(midi_folder, '9.mid'))
note_sequence = midi_to_note_sequence(midi_data)


In [16]:
text_sequence_converted = note_seq_to_text(note_sequence.notes)

In [17]:
text_sequence_converted

'PIECE_START TRACK_START INST=0 DENSITY=0 BAR_START NOTE_ON=72 TIME_DELTA=4.0 NOTE_OFF=72 NOTE_ON=72 TIME_DELTA=4.0 NOTE_OFF=72 NOTE_ON=68 TIME_DELTA=4.0 NOTE_OFF=68 NOTE_ON=70 TIME_DELTA=4.0 NOTE_OFF=70 BAR_END BAR_START NOTE_ON=72 TIME_DELTA=4.0 NOTE_OFF=72 NOTE_ON=73 TIME_DELTA=4.0 NOTE_OFF=73 NOTE_ON=72 TIME_DELTA=2.0 NOTE_OFF=72 NOTE_ON=70 TIME_DELTA=2.0 NOTE_OFF=70 NOTE_ON=69 TIME_DELTA=4.0 NOTE_OFF=69 BAR_END BAR_START NOTE_ON=70 TIME_DELTA=2.0 NOTE_OFF=70 NOTE_ON=69 TIME_DELTA=2.0 NOTE_OFF=69 NOTE_ON=67 TIME_DELTA=4.0 NOTE_OFF=67 NOTE_ON=65 TIME_DELTA=2.0 NOTE_OFF=65 NOTE_ON=67 TIME_DELTA=2.0 NOTE_OFF=67 NOTE_ON=68 TIME_DELTA=4.0 NOTE_OFF=68 BAR_END BAR_START NOTE_ON=67 TIME_DELTA=6.0 NOTE_OFF=67 NOTE_ON=65 TIME_DELTA=2.0 NOTE_OFF=65 NOTE_ON=65 TIME_DELTA=8.0 NOTE_OFF=65 BAR_END BAR_START NOTE_ON=72 TIME_DELTA=4.0 NOTE_OFF=72 NOTE_ON=72 TIME_DELTA=4.0 NOTE_OFF=72 NOTE_ON=70 TIME_DELTA=4.0 NOTE_OFF=70 NOTE_ON=68 TIME_DELTA=4.0 NOTE_OFF=68 BAR_END BAR_START NOTE_ON=63 TIME_DELTA=

In [18]:
# Reverting back to note sequence to visually compare:
sequence_new = token_sequence_to_note_sequence(text_sequence_converted, use_program=False)
note_seq.plot_sequence(sequence_new)
note_seq.play_sequence(sequence_new)

The end.


Loading BokehJS ...